# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [49]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [50]:
stocks = pd.read_csv('sp_500_stocks.csv')


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [51]:
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f"{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2550090793950, 'week52high': 156.59, 'week52low': 104.52, 'week52highSplitAdjustOnly': 155.83, 'week52lowSplitAdjustOnly': 104.8, 'week52change': 0.20907274368981613, 'sharesOutstanding': 17160200619, 'float': 0, 'avg10Volume': 82972068, 'avg30Volume': 72018592, 'day200MovingAvg': 134.68, 'day50MovingAvg': 146.81, 'employees': 150020, 'ttmEPS': 5.1, 'ttmDividendRate': 0.8570506457789137, 'dividendYield': 0.005768211112637187, 'nextDividendDate': '', 'exDividendDate': '2021-08-02', 'nextEarningsDate': '2021-10-15', 'peRatio': 28.692228663771427, 'beta': 1.54966352902767, 'maxChangePercent': 57.8612719917151, 'year5ChangePercent': 4.915660145293379, 'year2ChangePercent': 1.891228371000845, 'year1ChangePercent': 0.2711559386834346, 'ytdChangePercent': 0.1221896618517414, 'month6ChangePercent': 0.14966533526955347, 'month3ChangePercent': 0.1737388897473619, 'month1ChangePercent': 0.015726478579248852, 'day30ChangePercent': 0.02119820469023612, 'day

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [52]:
data['year1ChangePercent']

0.2711559386834346

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [53]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [54]:
def momentum_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = 'N/A'

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        data[symbol]['stats']['year1ChangePercent'],
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe

final_dataframe = momentum_data()
print(final_dataframe)

    Ticker   Price One-Year Price Return Number of Shares to Buy
0        A  171.92              0.760637                     N/A
1      AAL   19.10                   0.5                     N/A
2      AAP  213.86              0.357862                     N/A
3     AAPL  154.79              0.270201                     N/A
4     ABBV  121.73              0.312717                     N/A
..     ...     ...                   ...                     ...
500    YUM  137.11              0.454866                     N/A
501    ZBH  150.38              0.071933                     N/A
502   ZBRA  573.73              1.088446                     N/A
503   ZION   54.26              0.751082                     N/A
504    ZTS  212.66              0.317607                     N/A

[505 rows x 4 columns]


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [55]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50] # Dump all but the first 50
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.56,2.29299,N/A
1,272,KSS,57.79,2.109965,N/A
2,441,TPR,41.42,1.900298,N/A
3,106,COF,176.08,1.705769,N/A
4,344,NUE,119.27,1.643662,N/A
5,129,DFS,127.83,1.609651,N/A
6,253,IVZ,24.33,1.518748,N/A
7,23,ALB,224.91,1.502235,N/A
8,148,DVN,26.70,1.480716,N/A
9,453,UAA,24.24,1.455656,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [56]:
def portfolio_input():
    global portfolio_size

    portfolio_size = 1000000 # input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('Portfolio size should be an integer.')
        portfolio_size = input('Enter the value of your portfolio:')
        float(portfolio_size)

portfolio_input()
print(portfolio_size)

1000000


In [57]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    buy_count =  math.floor(position_size / final_dataframe.loc[i, 'Price'])
    final_dataframe.loc[i, 'Number of Shares to Buy'] = buy_count

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,83.56,2.29299,239
1,272,KSS,57.79,2.109965,346
2,441,TPR,41.42,1.900298,482
3,106,COF,176.08,1.705769,113
4,344,NUE,119.27,1.643662,167
5,129,DFS,127.83,1.609651,156
6,253,IVZ,24.33,1.518748,822
7,23,ALB,224.91,1.502235,88
8,148,DVN,26.70,1.480716,749
9,453,UAA,24.24,1.455656,825


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [58]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

def hq_momentum_data():
    dataframe = pd.DataFrame(columns = hqm_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        buy_count,
                        data[symbol]['stats']['year1ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month6ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month3ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month1ChangePercent'], 
                        np.nan,
                        np.nan,
                    ],
                    index = hqm_columns,
                    
                ),
                ignore_index = True
            )

    # https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float
    return dataframe.fillna(0)

hqm_dataframe = hq_momentum_data()
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,170.79,0.0,0.756973,0.0,0.336345,0.0,0.274387,0.0,0.127388,0.0,0.0
1,AAL,18.80,0.0,0.500000,0.0,-0.009961,0.0,-0.185616,0.0,-0.101179,0.0,0.0
2,AAP,217.67,0.0,0.353701,0.0,0.274287,0.0,0.071195,0.0,-0.002645,0.0,0.0
3,AAPL,153.32,0.0,0.264003,0.0,0.150318,0.0,0.166536,0.0,0.015834,0.0,0.0
4,ABBV,123.35,0.0,0.318433,0.0,0.159127,0.0,0.026014,0.0,0.024108,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.92,0.0,0.437302,0.0,0.290548,0.0,0.129226,0.0,0.160708,0.0,0.0
501,ZBH,148.58,0.0,0.070858,0.0,-0.072173,0.0,-0.130132,0.0,-0.067239,0.0,0.0
502,ZBRA,572.90,0.0,1.108249,0.0,0.167456,0.0,0.164288,0.0,0.100173,0.0,0.0
503,ZION,55.58,0.0,0.754897,0.0,0.056936,0.0,-0.051301,0.0,0.081169,0.0,0.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [59]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        column_return = f'{time_period} Price Return'
        column_percentile = f'{time_period} Return Percentile'

        # print(hqm_dataframe.convert_dtypes().dtypes)
        # print(hqm_dataframe[column_return])
        price_return_all = pd.to_numeric(hqm_dataframe[column_return].astype(float))
        price_return_single = hqm_dataframe.loc[row, column_return]

        percentile = score(price_return_all, price_return_single)
        hqm_dataframe.loc[row, column_percentile] = percentile

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,170.79,0.0,0.756973,86.138614,0.336345,88.118812,0.274387,95.643564,0.127388,93.663366,0.0
1,AAL,18.80,0.0,0.500000,65.544554,-0.009961,17.821782,-0.185616,4.356436,-0.101179,4.158416,0.0
2,AAP,217.67,0.0,0.353701,45.940594,0.274287,78.811881,0.071195,66.138614,-0.002645,30.891089,0.0
3,AAPL,153.32,0.0,0.264003,34.455446,0.150318,55.049505,0.166536,85.940594,0.015834,42.970297,0.0
4,ABBV,123.35,0.0,0.318433,41.782178,0.159127,57.623762,0.026014,51.881188,0.024108,46.930693,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.92,0.0,0.437302,58.217822,0.290548,81.782178,0.129226,78.811881,0.160708,96.831683,0.0
501,ZBH,148.58,0.0,0.070858,13.267327,-0.072173,10.099010,-0.130132,10.099010,-0.067239,11.287129,0.0
502,ZBRA,572.90,0.0,1.108249,95.643564,0.167456,59.603960,0.164288,85.148515,0.100173,87.524752,0.0
503,ZION,55.58,0.0,0.754897,85.940594,0.056936,33.267327,-0.051301,28.712871,0.081169,81.782178,0.0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [62]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,170.79,0.0,0.756973,86.138614,0.336345,88.118812,0.274387,95.643564,0.127388,93.663366,90.891089
1,AAL,18.80,0.0,0.500000,65.544554,-0.009961,17.821782,-0.185616,4.356436,-0.101179,4.158416,22.970297
2,AAP,217.67,0.0,0.353701,45.940594,0.274287,78.811881,0.071195,66.138614,-0.002645,30.891089,55.445545
3,AAPL,153.32,0.0,0.264003,34.455446,0.150318,55.049505,0.166536,85.940594,0.015834,42.970297,54.603960
4,ABBV,123.35,0.0,0.318433,41.782178,0.159127,57.623762,0.026014,51.881188,0.024108,46.930693,49.554455
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.92,0.0,0.437302,58.217822,0.290548,81.782178,0.129226,78.811881,0.160708,96.831683,78.910891
501,ZBH,148.58,0.0,0.070858,13.267327,-0.072173,10.099010,-0.130132,10.099010,-0.067239,11.287129,11.188119
502,ZBRA,572.90,0.0,1.108249,95.643564,0.167456,59.603960,0.164288,85.148515,0.100173,87.524752,81.980198
503,ZION,55.58,0.0,0.754897,85.940594,0.056936,33.267327,-0.051301,28.712871,0.081169,81.782178,57.425743


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [61]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

NameError: name 'writer' is not defined

## Saving Our Excel Output

As before, saving our Excel output is very easy: